<a href="https://colab.research.google.com/github/KrishaManandhar/Extract-Transform-Load-Log/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Mounted drive for accessing files from there
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
import glob                           # this module helps in selecting files
import pandas as pd                   # this module helps in processing csv files
import xml.etree.ElementTree as ET    # this module helps in processing xml files
from datetime import datetime

In [13]:
tmpfile = "temp.tmp"                  # file used to store all extracted data
logfile = "logfile.txt"               # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

# CSV Extract Function

In [18]:
# read a comma-separated values(csv) file into dataframe
def extracting_from_csv(file_to_process):     
  dataframe = pd.read_csv(file_to_process)   
  return dataframe

# JSON Extract Function

In [19]:
# reading data from json file
def extracting_from_json(file_to_process):          
  dataframe = pd.read_json(file_to_process,lines=True) # lines: True helps when there is presence of \n in json otherwise it will give trailing error
  return dataframe

# XML Extract Function

In [20]:
# reading xml data
def extracting_from_xml(file_to_process):
  dataframe = pd.DataFrame(columns=["name","height","weight"])
  tree = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:       # dictionary making
    name = person.find("name").text
    height = float(person.find("height").text)
    weight = float(person.find("weight").text)
    dataframe = dataframe.append({"name":name,"height":height,"weight":weight}, ignore_index = True)  # Here ignore_index=True is necessary to append  dictionary to dataframe
  return dataframe 

# Extract Function

In [22]:
def extract():
  extracted_data = pd.DataFrame(columns=['name','height','weight'])   # making empty dataframe with name, height, weight columns

  for csvfile in glob.glob("/content/gdrive/MyDrive/3. ExtractTransformLoadLog HandsOn/*.csv"):   #selecting all csv files using glob
    extracted_data = extracted_data.append(extracting_from_csv(csvfile), ignore_index = True)     #adding every csv file in extracted_data
  
  for jsonfile in glob.glob("/content/gdrive/MyDrive/3. ExtractTransformLoadLog HandsOn/*.json"):
    extracted_data = extracted_data.append(extracting_from_json(jsonfile),ignore_index = True)

  for xmlfile in glob.glob("/content/gdrive/MyDrive/3. ExtractTransformLoadLog HandsOn/*.xml"):
    extracted_data = extracted_data.append(extracting_from_xml(xmlfile), ignore_index = True)
  
  return extracted_data

# Transfrom Function

In [23]:
def transform(data):
  #Convert datatype of column to float
  data.height = data.height.astype(float)
  #Convert height in inches to millimeter
  data['height'] = round(data.height*0.0254,2)

  #Convert datatype of column to float
  data.weight = data.weight.astype(float)
  #Convert weight in pounds to kilograms
  data['weight'] = round(data.weight*0.0254,2)

  return data

# Load Function

In [24]:
def load(targetfile, data_to_load):   
  data_to_load.to_csv(targetfile)   # exporting dataframe to CSV format

# Log Function

In [25]:
# Keeping track of events happening
def log(message):
  timestamp_format = '%Y-%h-%d-%H-:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
  now = datetime.now()                    # get current timestamp
  timestamp = now.strftime(timestamp_format)
  print(timestamp + ',' + message + '\n')

# Running ETL process

In [26]:
log("ETL Job Started")

2022-Nov-15-14-:58:46,ETL Job Started



In [27]:
log("Extract phase started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

2022-Nov-15-14-:59:54,Extract phase started

2022-Nov-15-14-:59:55,Extract phase Ended



,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [28]:
log("Transform phase started")
transformed_data = transform(extracted_data)
log("Transform phase ended")
transformed_data

2022-Nov-15-15-:00:15,Transform phase started

2022-Nov-15-15-:00:15,Transform phase ended



,name,height,weight
0,alex,1.67,2.87
1,ajay,1.82,3.47
2,alice,1.76,3.89
3,ravi,1.73,3.62
4,joe,1.72,3.67
5,alex,1.67,2.87
6,ajay,1.82,3.47
7,alice,1.76,3.89
8,ravi,1.73,3.62
9,joe,1.72,3.67


In [29]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

2022-Nov-15-15-:00:26,Load phase Started

2022-Nov-15-15-:00:26,Load phase Ended

